In [2]:
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [3]:
import pandas as pd
import numpy as np

def getSamples(filename):
    data = pd.read_csv(filename, sep='\t')
    return data.values[:,1:].transpose()

data1 = getSamples("GSE2034-Normal-train.txt")
data2 = getSamples("GSE2034-Tumor-train.txt")

data1Label = np.zeros(len(data1)).reshape((-1, 1))
data2Label = np.ones(len(data2)).reshape((-1, 1))
x = np.concatenate((data1, data2))
y = np.concatenate((data1Label, data2Label))

In [4]:
dim = 12634
class Res1d(nn.Module):
    # the conv layers
    def __init__(self, inSize, outSize, kernel=(3,), strides=1,):
        super(Res1d, self).__init__()
        # hard-coded to do the padding correctly
        if inSize in (16,64,128,512) and strides is 2:
            pding = 0
        else:
            pding = 1
        self.l1 = nn.Conv1d(inSize, outSize, kernel, stride=strides, padding=pding, bias=False)
        self.l2 = nn.InstanceNorm1d(outSize)
        
        if strides > 1 or inSize != outSize:
            if strides > 1:
                self.r1 = nn.Identity()
                self.r2 = nn.AvgPool1d(strides)
            else:
                self.r1 = None
                self.r2 = None
            self.r3 = nn.Conv1d(inSize, outSize, 1, bias=False)
            self.r4 = nn.InstanceNorm1d(outSize)
    
        self.relu = nn.ReLU()
        
    def forward(self, x):
        l = self.l1(x)
        l = self.l2(l)
        
        if self.r1 is not None:
            r = self.r1(x)
            r = self.r2(r)
            r = self.r3(r)
            r = self.r4(r)
        else:
            r = self.r3(x)
            r = self.r4(r)
            
#         print(self.l1, l.shape, r.shape)
        x = l + r
        return self.relu(x)
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(dim, 64)
        self.l2 = nn.ReLU()
        
        self.r1 = Res1d(1, 4, 3)
        
        self.r2 = Res1d(4, 8, 3)
        self.r3 = Res1d(8, 8, 3, strides=2)
        
        self.r4 = Res1d(8, 16, 3)
        self.r5 = Res1d(16, 16, 3, strides=2)
        
        self.r6 = Res1d(16, 32, 3)
        self.r7 = Res1d(32, 32, 3, strides=2)
        
        self.r8 = Res1d(32, 64, 3)
        self.r9 = Res1d(64, 64, 3, strides=2)
        
        self.r10 = Res1d(64, 128, 3)
        self.r11 = Res1d(128, 128, 3, strides=2)
        
        self.r12 = Res1d(128, 256, 3)
        self.r13 = Res1d(256, 256, 3, strides=2)
        
        self.r14 = Res1d(256, 512, 3)
        self.r15 = Res1d(512, 512, 3, strides=2)
        
        self.r16 = Res1d(512, 1024, 3)
        self.r17 = Res1d(1024, 1024, 3, strides=2)
        
        # size is by experiment and hardcode
        self.lastLinear = nn.Linear(50240,32)
        self.lastRelu = nn.ReLU()
        self.lastAgg = nn.Linear(32,1)
        self.lastSigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # shape is (batch, channel, time)
        l = x
        l = self.l2(self.l1(l))
        
        # conv layers should operate on time
        r = x
        r = self.r4(self.r3(self.r2(self.r1(r))))
        r = self.r8(self.r7(self.r6(self.r5(r))))
        r = self.r12(self.r11(self.r10(self.r9(r))))
        r = self.r16(self.r15(self.r14(self.r13(r))))
        r = self.r17(r)
        
        # flatten l
        r = r.view(x.shape[0],1, -1)
        y = torch.cat((l,r),-1)
        y = self.lastLinear(y)
        y = self.lastRelu(y)
        y = self.lastAgg(y)
        y = self.lastSigmoid(y)
        return y

In [5]:
device = torch.device("cuda")
net = Net().to(device)


In [6]:
input_test = torch.randn(80, 1, dim).float().cuda()
out = net(input_test).cuda()
out.shape

torch.Size([80, 1, 1])

# Train the model

In [7]:
# get the data into (batch, channel = 1, length=dim)
data_torch = torch.from_numpy(x).view([-1, 1, dim]).float()
label_torch = torch.from_numpy(y).view([-1,1,1]).float()

In [8]:
import torch.optim as optim
criterion = nn.BCELoss() # Binary Cross Entropy 
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
for batch in range(100):  # loop over the dataset multiple times
    # get the inputs; data is a list of [inputs, labels]
    indices = np.random.choice(len(x), size=(30))
    inputs = x[indices]
    labels = y[indices]
    
    inputs = torch.from_numpy(inputs).view([-1, 1, dim]).float().cuda()
    labels = torch.from_numpy(labels).view([-1, 1]).float().cuda()
    
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs).view([-1,1]).cuda()
    loss = criterion(outputs, labels).cuda()
    loss.backward()
    optimizer.step()
    
    #if batch % 10 == 0:
    print(batch, "% Trained")

print('Finished Training')

0 % Trained
1 % Trained
2 % Trained
3 % Trained
4 % Trained
5 % Trained
6 % Trained
7 % Trained
8 % Trained
9 % Trained
10 % Trained
11 % Trained
12 % Trained
13 % Trained
14 % Trained
15 % Trained
16 % Trained
17 % Trained
18 % Trained
19 % Trained
20 % Trained
21 % Trained
22 % Trained
23 % Trained
24 % Trained
25 % Trained
26 % Trained
27 % Trained
28 % Trained
29 % Trained
30 % Trained
31 % Trained
32 % Trained
33 % Trained
34 % Trained
35 % Trained
36 % Trained
37 % Trained
38 % Trained
39 % Trained
40 % Trained
41 % Trained
42 % Trained
43 % Trained
44 % Trained
45 % Trained
46 % Trained
47 % Trained
48 % Trained
49 % Trained
50 % Trained
51 % Trained
52 % Trained
53 % Trained
54 % Trained
55 % Trained
56 % Trained
57 % Trained
58 % Trained
59 % Trained
60 % Trained
61 % Trained
62 % Trained
63 % Trained
64 % Trained
65 % Trained
66 % Trained
67 % Trained
68 % Trained
69 % Trained
70 % Trained
71 % Trained
72 % Trained
73 % Trained
74 % Trained
75 % Trained
76 % Trained
77 % Trai

In [10]:
# MPC learning
net

Net(
  (l1): Linear(in_features=12634, out_features=64, bias=True)
  (l2): ReLU()
  (r1): Res1d(
    (l1): Conv1d(1, 4, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (l2): InstanceNorm1d(4, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (r3): Conv1d(1, 4, kernel_size=(1,), stride=(1,), bias=False)
    (r4): InstanceNorm1d(4, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (relu): ReLU()
  )
  (r2): Res1d(
    (l1): Conv1d(4, 8, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (l2): InstanceNorm1d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (r3): Conv1d(4, 8, kernel_size=(1,), stride=(1,), bias=False)
    (r4): InstanceNorm1d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (relu): ReLU()
  )
  (r3): Res1d(
    (l1): Conv1d(8, 8, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
    (l2): InstanceNorm1d(8, eps=1e-05, momentum=0.1, affine=False, track_runnin

In [14]:
net(data_torch.cuda())

tensor([[[5.5693e-03]],

        [[1.2892e-02]],

        [[3.3465e-03]],

        [[4.2194e-04]],

        [[4.5199e-03]],

        [[5.4603e-03]],

        [[4.1989e-03]],

        [[1.0753e-02]],

        [[5.1220e-03]],

        [[5.5587e-03]],

        [[3.8188e-03]],

        [[2.6553e-03]],

        [[4.9701e-03]],

        [[3.5327e-03]],

        [[9.2005e-03]],

        [[5.1572e-03]],

        [[2.5510e-03]],

        [[4.8186e-03]],

        [[6.3429e-03]],

        [[2.2609e-03]],

        [[1.8099e-03]],

        [[5.7676e-03]],

        [[3.3225e-03]],

        [[4.9803e-03]],

        [[3.2558e-03]],

        [[1.9185e-03]],

        [[1.9541e-03]],

        [[7.1851e-03]],

        [[3.5984e-03]],

        [[7.4177e-03]],

        [[2.3416e-02]],

        [[6.9360e-03]],

        [[3.9892e-03]],

        [[1.5723e-02]],

        [[5.1656e-03]],

        [[4.1250e-03]],

        [[8.3235e-03]],

        [[2.9352e-03]],

        [[2.8757e-03]],

        [[4.7430e-03]],

